In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip freeze > requirements2.txt

In [ ]:
!pip install -q -U transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.

In [ ]:
!pip install trl==0.7.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.9 MB/s eta 0:00:00


## Dataset



In [ ]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset_name = 'Lennard-Heuer/DAIM-LLM'
data_files = 'fine_tuning_data (14).jsonl'
# data_files = 'Long Exemplary Instruction Tasks.jsonl'  # Or 'traini.jsonl' based on the correct file name
dataset_train = load_dataset(dataset_name, data_files=data_files)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# for reverence: from datasets import load_dataset

# for reverence:  #dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

# for reverence:  dataset_name = 'AlexanderDoria/novel17_test' #french novels
# for reverence:  dataset = load_dataset(dataset_name, split="train")

In [ ]:
print(dataset_train)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# Define the model name
model_name = "meta-llama/Meta-Llama-3-8B"

# Load the configuration
config = AutoConfig.from_pretrained(model_name)

# Modify the max_position_embeddings to 4096
config.max_position_embeddings = 4096

# Now, load the model with the modified configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    trust_remote_code=True
)

# Update model configuration to not use cache if needed
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

## Loading the model

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Old stuff

In [ ]:
# from peft import LoraConfig, get_peft_model

# lora_alpha = 16
# lora_dropout = 0.1
# lora_r = 64

#peft_config = LoraConfig(
#    lora_alpha=lora_alpha,
#    lora_dropout=lora_dropout,
#    r=lora_r,
#    bias="none",
#    task_type="CAUSAL_LM"
#)

In [ ]:
n_freeze = 28 # you can play with this parameter


# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[n_freeze:].parameters(): param.requires_grad = True

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

used to be 4/4

In [ ]:
pip install accelerate -U transformers[torch]

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"
max_seq_length = 512

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
train_dataset = dataset_train['train']

In [ ]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 106
})

In [ ]:
# Upgrade the trl package to the latest version
!pip install --upgrade trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.7.11
    Uninstalling trl-0.7.11:
      Successfully uninstalled trl-0.7.11


In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,11.365100
20,4.460600
30,4.050700
40,2.937700
50,2.148900
60,3.507500
70,2.934200
80,2.434400
90,2.350300
100,2.156500


TrainOutput(global_step=100, training_loss=3.834604263305664, metrics={'train_runtime': 421.7131, 'train_samples_per_second': 0.237, 'train_steps_per_second': 0.237, 'total_flos': 123290902511616.0, 'train_loss': 3.834604263305664, 'epoch': 0.9433962264150944})

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)



In [ ]:
# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Load the model and tokenizer from the saved directory
model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Push to Hugging Face Hub
model.push_to_hub("Lennard-Heuer/Llama3-FT_V4")
tokenizer.push_to_hub("Lennard-Heuer/Llama3-FT_V4")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00003-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Lennard-Heuer/Llama3-FT_V4/commit/ea0d0f569951e019206202a822822fc32e62d6c8', commit_message='Upload tokenizer', commit_description='', oid='ea0d0f569951e019206202a822822fc32e62d6c8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Trainer

# Assuming `trainer` is your Trainer instance with the model you've fine-tuned

# Save the model; this accounts for parallel/distributed training scenarios
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputsV3")

# Assuming you've already logged in to Hugging Face in your environment
# If not, you would need to log in using `huggingface_hub.login()`

# Push to the Hub. Replace "your_model_name" with your desired model name on the Hub
# Ensure the model name is unique and descriptive enough
trainer.push_to_hub("your_model_name")


In [ ]:
trainer.model.save_pretrained("outputsV4")


In [ ]:
from huggingface_hub import login
login()

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [ ]:
trainer.push_to_hub("Lennard-Heuer/Llama3-FT-D")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-66646ebd-7af465466d0890cf617e5df5;97812664-2042-41ff-a46c-9917a8e1f19f)

Invalid username or password.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputsV3")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputsV3')
model = get_peft_model(model, lora_config)

Not in the 1littlecoder skript

In [ ]:
from transformers import AutoModel

# Assuming 'your_library' is where LoraConfig is from

# Load the base model
model = AutoModel.from_pretrained('outputsV3')

# Load the adapter configuration
# The actual function to load the adapter will depend on the library you are using
# Replace 'load_adapter_function' with the actual function name
model.load_adapter('outputsV3/adapter_config.json')

# Load the LoRA configuration
lora_config = LoraConfig.from_pretrained('outputsV3')

# Apply the LoRA configuration to the model
# This will depend on how the LoRA configuration is applied in your library
# Replace 'apply_lora_config_to_model' with the actual function you use to apply the configuration
model = get_peft_model(model, lora_config)


OSError: outputsV3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
lora_config = LoraConfig.from_pretrained('outputsV3')
model = get_peft_model(model, lora_config)

In [ ]:
train_dataset['text']

['### Human: Which basic model and which extension(s) of the basic model should I use? You must return one or more of the 8 models, while combinations are also possible: Which production planning model from the book factory physics would be the most appropriate and why? The possible answers are: Aggregate Planning (AP), Single Product, I. Basic model; Aggregate Planning (AP), Product Mix Planning II. Basic model; Aggregate Planning (AP), Product Mix Planning III. Capacitated resources; Aggregate Planning (AP), Product Mix Planning IV. Utilization Matching; Aggregate Planning (AP), Product Mix Planning V. Backorders; Aggregate Planning (AP), Product Mix Planning VI. Overtime; Aggregate Planning (AP), Product Mix Planning VII. Yield loss; Workforce Planning (WP), VIII. Basic model., Input: labor_type = mix of short-term contract and full-time employees, capacity_constraints = limited machinery during peak seasons, variable_costs = energy and raw materials, workload_variation = high durin

In [ ]:
text = "Mention a production planning model"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Mention a production planning model and the first thing that comes to mind is the MRP II model. nobody is going to argue that the MRP II model is not a good model. It is a good model. It is a good model for a lot of companies. But


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("llama2-qlora-finetunined-XXX-1")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Lennard-Heuer/llama2-qlora-finetunined-XXX/commit/e4708eac073d584bf441b04da74f949130f47231', commit_message='Upload model', commit_description='', oid='e4708eac073d584bf441b04da74f949130f47231', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
Lennard-Heuer/llama2-qlora-finetunined-XXX

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "Lennard-Heuer/llama2-qlora-finetunined-XXX"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

OSError: Can't load tokenizer for 'Lennard-Heuer/llama2-qlora-finetunined-XXX'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Lennard-Heuer/llama2-qlora-finetunined-XXX' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.

In [ ]:
from transformers import AutoModel, AutoConfig
import json
import safetensors.torch

# Load the base pre-trained LLAMA2 model
model = AutoModel.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded")

# Load adapter configuration
with open("Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_config.json", "r") as file:
    adapter_config = json.load(file)

# Assuming the library and model support loading adapters in this way
# This part is more conceptual since actual implementation can vary
model.load_adapter("Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_model.safetensors", config=adapter_config)

# If safetensors is the format for your adapter_model, ensure you have a method to load it
# For example, using safetensors.torch.load if you're working with PyTorch
adapter_weights = safetensors.torch.load("Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_model.safetensors")
model.load_state_dict(adapter_weights, strict=False)

# Your model is now ready to use with the loaded adapters


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_config.json'

In [ ]:
from transformers import AutoModel, AutoConfig
import json
import safetensors.torch

# Load the base pre-trained LLAMA2 model
model = AutoModel.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded")

# Load adapter configuration
# with open("Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_config.json", "r") as file:
    # adapter_config = json.load(file)


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
import torch
import transformers
import peft
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
!pip install -q peft transformers torch

In [ ]:
peft_model_id = "Lennard-Heuer/llama2-qlora-finetunined-XXX"

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules={'q_proj', 'v_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [ ]:
batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 “Training models with PEFT and LoRa is cool” ->: 100%
“Training models with PEFT and LoRa is cool” ->: 100%
“Training models with PEFT and LoRa is cool” ->: 99%



In [ ]:
config =

In [ ]:
import torch
import peft

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

with open("Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_config.json", "r") as file:
    adapter_config = json.load(file)


FileNotFoundError: [Errno 2] No such file or directory: 'Lennard-Heuer/llama2-qlora-finetunined-XXX/adapter_config.json'

In [ ]:
# Assuming `adapter_config` is loaded from `adapter_config.json`
# and `adapter_weights` from `adapter_model.safetensors`

# Load adapter configuration
with open("path_to/adapter_config.json", "r") as file:
    adapter_config = json.load(file)

# Assuming you have a mechanism to load safetensors into a format compatible with your model
# This might involve using a custom loading function or converting safetensors to PyTorch tensors
adapter_weights = safetensors.torch.load("path_to/adapter_model.safetensors")

# Integrate adapter configuration and weights into the model
# This step is highly dependent on the specific mechanisms your adapter and model support
# For example, with PEFT and LoRA, you might need to use specific functions to inject the adapter
model = get_peft_model(model, adapter_config, peft_config, adapter_weights)


In [ ]:
text = "Mention a production planning model"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Mention a production planning model and the first thing that comes to mind is the MRP II model. nobody can deny that the MRP II model is the most widely used production planning model in the world.
The MRP II model is a production planning model that is


**New Section**

In [ ]:
!pip freeze > requirements2.txt

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q peft transformers torch

In [ ]:
peft_model_id = "Lennard-Heuer/llama2-qlora-finetunined-XXX"

In [ ]:
import torch
import transformers
import peft
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import accelerate
import bitsandbytes

In [ ]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
pip install accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Lennard-Heuer/llama2-qlora-finetunined-XXX"
config = PeftConfig.from_pretrained(peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
!pip install -q peft transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 22.2 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
import peft
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
peft_model_id = "Lennard-Heuer/llama2-qlora-finetunined-XXX"

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]